In [6]:
# This is a "never submitted script" for the RSNA BRAIN TUMOR
# CLASSIFICATION - Kaggle competition.
#
# I gave up on this project because I could not create a model
# that seemed to learn anything from the data. But when I saw
# the final results and understood that in the end nobody really
# could, I decided to give it a try in a "late submission".
#
# Unfortunatly I spent too much time on other projects and the
# time window allowing late submissions closed before I could make
# that submission.
#
# That's too bad because I had created a complexe (or messy^^)
# process that succeded more or less in identifying and extracting
# the tumor zone in the 3d images. And it would have been interesting
# to know if it helped or not in this complicated competition.


#####################
# Python librairies #
#####################

import numpy as np
import pandas as pd
import os
import shutil
import pickle
import cv2
import matplotlib.pylab as plt
import matplotlib.patches as patches
import pickle

import nibabel as nib
import imgaug.augmenters as iaa
import pydicom as dicom
import SimpleITK as sitk
import random

from glob import glob
from time import time
from PIL import Image
from scipy.ndimage.filters import gaussian_filter
from tqdm import tqdm
from numpy.random import seed

from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Conv3D, Input, BatchNormalization,\
    Reshape, MaxPool3D, Dense, Dropout, Activation, Conv2D,\
    GlobalMaxPool2D, MaxPool2D, Concatenate, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.random import set_seed

from IPython.display import FileLink
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score,\
    plot_roc_curve

sitk.ProcessObject_SetGlobalWarningDisplay(False)

seed(48)
set_seed(48)

In [9]:
#########################
# Constants définitions #
#########################

INPUT = "/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/"
OUT = "/kaggle/working"
TRAIN_PATH = INPUT + "/" + sorted(os.listdir(INPUT))[2]
TEST_PATH = INPUT + "/" + sorted(os.listdir(INPUT))[1]
OUT_TRAIN = "train"
OUT_VAL = "val"
OUT_TEST = "test"

random.seed(48)

# Preprocessing parameters
NB_SAMPLES = 12  # number of kept images PER MODALITY
SIZE_TEMP = (512, 512)  # size in the 3d images
SIZE = (150, 150)  # size of the final 3d images for medelization
MODS = sorted(["FLAIR", "T1w", "T1wCE", "T2w"])  # list of used modality
NB_MOD = len(MODS)
THRESHOLD = 220  # threshold pixel value when filtering for finding zoi
RABOTAGE = 40  # number of pixel on the image sides we get rid of
ZOOM = 1.2  # zoom image transformation parameter
BATCH = 8

# Data generator parameters
LIST_ITERATION = 7  # number of iterations per epochs for the generator

# Define our reference image for the "resample" function
reference_image_path = TRAIN_PATH + "/00143/T1w"
reader = sitk.ImageSeriesReader()
dicom_names = reader.GetGDCMSeriesFileNames(reference_image_path)
reader.SetFileNames(dicom_names)
ref_image = reader.Execute()

In [49]:
##########################################
# Preprocessing & modelization functions #
##########################################

def pertinent_slices(im3d, seuil=0.6):
    """
    Find if the image brain is important enough.
    """
    for i in range(im3d.shape[0]):

        if np.mean(im3d[i]) != 0:

            mk = im3d[i] > 0

            coords = np.argwhere(mk)
            y0, _ = coords.min(axis=0)
            y1, _ = coords.max(axis=0) + 1

            haut = y1-y0

            if haut/SIZE_TEMP[0] < seuil:

                im3d[i, :, :] = 0

    return im3d


def zoom(im3d):
    """
    This function apply a zoom transformation to each slice of a 3d image
    """
    scale = iaa.Affine(scale=(ZOOM))

    new_image = np.zeros(im3d.shape)

    for i in range(im3d.shape[0]):

        img = im3d[i, :, :]
        img = scale.augment_image(img)
        new_image[i] = img

    return new_image


def rabote(image, rabotage=RABOTAGE):
    """
    Get rid of the edges of brain image where we often find bright pixels.
    """
    temp_image = image.copy()

    for h in range(temp_image.shape[0]):

        for i in range(temp_image.shape[1]):

            if np.mean(temp_image[h, i, :]) != 0:

                vector = temp_image[h, i, :]
                temp_mask = vector > 0
                temp_coords = np.argwhere(temp_mask)

                mini = temp_coords.min(axis=0)
                maxi = temp_coords.max(axis=0)
                temp_image[h, i, : int(mini + rabotage/2)] = 0
                temp_image[h, i, int(maxi-rabotage/2):] = 0

    for h in range(temp_image.shape[0]):

        for i in range(temp_image.shape[2]):

            if np.mean(temp_image[h, :, i]) != 0:

                vector = temp_image[h, :, i]
                temp_mask = vector > 0
                temp_coords = np.argwhere(temp_mask)

                mini = temp_coords.min(axis=0)
                maxi = temp_coords.max(axis=0)
                temp_image[h, : int(mini + rabotage), i] = 0
                temp_image[h, int(maxi-rabotage):, i] = 0

    return temp_image


def zoi_cube(flair_image, ID, threshold=THRESHOLD, rabotage=RABOTAGE):
    """
    Function that tries to identify the tumoral zone in the 3d images.
    It returns a "zoi".
    """
    bool_zoom = False
    temp_image = flair_image.copy()
    temp_image = pertinent_slices(temp_image)

    if np.mean(temp_image) == 0:

        temp_image = flair_image.copy()
        temp_image = zoom(temp_image)
        temp_image = pertinent_slices(temp_image)
        bool_zoom = True

    if np.mean(temp_image) == 0:

        temp_image = flair_image.copy()
        temp_image = pertinent_slices(temp_image, 0.5)
        bool_zoom = False

    temp_image = rabote(temp_image)

    masked = np.ma.masked_equal(temp_image, 0)
    mean_image = np.mean(masked, axis=0).astype(np.uint8)

    eroded = cv2.erode(mean_image, None, iterations=6)
    eroded = normalize_img(eroded).astype(np.uint8)
    mask = eroded > threshold

    coords = np.argwhere(mask)

    y0, x0 = coords.min(axis=0)
    y1, x1 = coords.max(axis=0) + 1

    x_center = x0 + int(0.5*(x1-x0))
    y_center = y0 + int(0.5*(y1-y0))

    x_min = x_center - int(SIZE[0]/2)
    x_max = x_center + int(SIZE[0]/2)
    y_min = y_center - int(SIZE[0]/2)
    y_max = y_center + int(SIZE[0]/2)

    # cas limites
    if x_min < 0:

        x_min = 0
        x_max = SIZE[0]

    elif x_max > SIZE_TEMP[0]-1:

        x_min = SIZE_TEMP[0]-SIZE[0]
        X_max = SIZE_TEMP[0]

    elif y_min < 0:

        y_min = 0
        y_max = SIZE[0]

    elif y_max > SIZE_TEMP[0]-1:

        y_min = SIZE_TEMP[0]-SIZE[0]
        y_max = SIZE_TEMP[0]

    temp_image_filtered = normalize_img(temp_image).astype(np.uint8)
    temp_image_filtered = temp_image_filtered[:, y_min:y_max, x_min:x_max]\
        .astype(np.uint8)

    if np.max(temp_image_filtered) <= threshold:

        temp_image_filtered = normalize_img(temp_image_filtered)

    msk = temp_image_filtered >= threshold
    coor = np.argwhere(msk)

    z0, _, _ = coor.min(axis=0)
    z1, _, _ = coor.max(axis=0) + 1

    zoi = (x_min, x_max, y_min, y_max, z0, z1, bool_zoom)

    return zoi


def create_dataframe(path, mods):
    """
    Create a reference dataframe.
    """
    chemins = []
    patients = []
    scans = []

    liste = sorted(glob(path + "/*/*"))

    for elt in liste:

        chemins.append(elt)
        patients.append(elt.split("/")[-2])
        scans.append(elt.split("/")[-1])

    df = pd.DataFrame(list(zip(chemins, patients, scans)),
                      columns=["chemins", "BraTS21ID", "scan type"])

    df = df[df["scan type"].isin(mods)]

    return df


def filter_train_df(df):
    """
    Get rid of bad data.
    """
    liste_del = ['00123', '00109', '00709']

    for k in liste_del:
        df = df.loc[df["BraTS21ID"] != k]

    return df


def create_dicts(path_to_csv):
    """
    Create a dict object with which we'll later build our labels.
    """
    df_labels = pd.read_csv(path_to_csv)
    dico_labels = {}

    for r in df_labels.iterrows():
        dico_labels[str(r[1][0])] = str(r[1][1])

    return dico_labels


def zoi_crop(img3d, zoi):
    """
    We use the "zoi" determined sooner to crop the 3d image.
    """
    slices = zoi[5] - zoi[4]
    bool_zoom = zoi[6]

    if slices < NB_SAMPLES:

        missing = NB_SAMPLES-slices

        extension = int(missing//2)
        lim_haute = zoi[4] - extension

        if missing % 2 == 0:

            lim_basse = zoi[5] + extension

        else:

            lim_basse = zoi[5] + extension + 1

        if lim_haute < 0:

            lim_haute = 0
            lim_basse = NB_SAMPLES

        if lim_basse > img3d.shape[0] - 1:

            lim_basse = img3d.shape[0] - 1
            lim_haute = img3d.shape[0] - (NB_SAMPLES + 1)

    elif slices > NB_SAMPLES:

        surplus = slices - NB_SAMPLES
        reduction = int(surplus//2)
        lim_haute = zoi[4] + reduction

        if surplus % 2 == 0:

            lim_basse = zoi[5] - reduction

        else:

            lim_basse = zoi[5] - (reduction + 1)

    else:

        lim_haute = zoi[4]
        lim_basse = zoi[5]

    if bool_zoom:

        img3d = zoom(img3d)

    img3d = img3d[lim_haute: lim_basse, zoi[2]:zoi[3], zoi[0]:zoi[1]]

    return img3d.astype(np.uint8)


def resize_3d(img3d, size):
    """
    Resize 3d images.
    """
    sh = img3d.shape
    new_image_3D = np.zeros((sh[0], size[0], size[1]))

    for i in range(sh[0]):

        im = Image.fromarray(img3d[i, :, :])
        im = im.convert("L")
        im = im.resize((size))
        im = np.array(im)
        new_image_3D[i] = im

    return new_image_3D


def crop_center(img3d):
    """
    Center-crop 3d images that are not square.
    """
    crop = iaa.size.CropToSquare(position="center")

    mini = min(img3d.shape[1], img3d.shape[2])
    new_image_3D = np.zeros((img3d.shape[0], mini, mini))

    for i in range(img3d.shape[0]):

        temp = img3d[i, :, :]
        temp = crop.augment_image(temp)
        new_image_3D[i] = temp

    return new_image_3D


def normalize_img(img3d):
    """
    Normalize 3d images.
    """
    img3d = img3d - np.min(img3d)
    if np.max(img3d) != 0:
        img3d = img3d / np.max(img3d)
    img3d = (img3d * 255).astype(np.uint8)
    return img3d


def resample(image3D, ref_image):
    """
    Re-orient 3d images according to the reference image.
    """
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(ref_image)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetTransform(sitk.AffineTransform(ref_image.GetDimension()))
    resampler.SetOutputSpacing(ref_image.GetSpacing())
    resampler.SetSize(ref_image.GetSize())
    resampler.SetOutputDirection(ref_image.GetDirection())
    resampler.SetOutputOrigin(ref_image.GetOrigin())
    resampler.SetDefaultPixelValue(image3D.GetPixelIDValue())
    resampled_image_3D = resampler.Execute(image3D)

    return resampled_image_3D


def preprocessing(df_source, target, mods=MODS):
    """
    Whole preprocessing function.
    """
    start = time()

    if os.path.isdir(target):
        shutil.rmtree(target)
        os.mkdir(target)

    else:
        os.mkdir(target)

    liste_IDs = sorted(set(df_source["BraTS21ID"].values.tolist()))
    target_path = target

    for ix, ID in enumerate(liste_IDs):

        if ix != 0:
            if ix % 50 == 0:
                print(f"Traitement {ix}ème patient au bout de\
                {time()-start:.0f} secondes")

        dim_ID = str(int(ID))
        target_name = dim_ID

        image_3D = np.zeros((NB_SAMPLES * NB_MOD, SIZE[0], SIZE[1]))
        zoi = ()

        for i, mod in enumerate(mods):

            row = df_source[(df_source["BraTS21ID"] == ID) &
                            (df_source["scan type"] == mod)]

            path_dcm = row["chemins"].values[0]
            reader = sitk.ImageSeriesReader()
            dicom_names = reader.GetGDCMSeriesFileNames(path_dcm)
            reader.SetFileNames(dicom_names)
            im3D = reader.Execute()
            new_img3D = resample(im3D, ref_image)
            array_3D = normalize_img(sitk.GetArrayFromImage(new_img3D))

            if array_3D.shape[1] != array_3D.shape[2]:

                array_3D = crop_center(array_3D)

            array_3D = resize_3d(array_3D, SIZE_TEMP).astype(np.uint8)

            if mod == "FLAIR":

                zoi = zoi_cube(array_3D, ID)

            array_3D = zoi_crop(array_3D, zoi)
            image_3D[i*NB_SAMPLES:(i+1)*NB_SAMPLES, :, :] = array_3D

        image_3D = normalize_img(image_3D).astype(np.uint8)
        np.save(target_path + "/" + target_name, image_3D)

In [83]:
####################################
# Data generator with augmentation #
####################################

class DataGenerator(Sequence):
    # Generates data
    def __init__(self,
                 list_IDs,
                 labels,
                 source,
                 batch_size,
                 nb_samples,
                 moda,
                 shuffle=True,
                 divise=True,
                 aug=True,
                 train=True):

        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.nb_samples = nb_samples
        self.moda = moda
        self.shuffle = shuffle
        self.source = source
        self.divise = divise
        self.aug = aug
        self.train = train
        self.on_epoch_end()

    def __len__(self):

        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):

        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        if self.train:

            y = [int(self.labels[k]) for k in indexes]
            y = np.array(y)

            X = self.__data_generation(list_IDs_temp)

            return X, y

        else:

            X = self.__data_generation(list_IDs_temp)

            return X

    def on_epoch_end(self):

        self.indexes = np.arange(len(self.list_IDs))

        if self.shuffle:

            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):

        if self.divise:

            X = np.empty((self.batch_size, len(self.moda), self.nb_samples,
                          SIZE[0], SIZE[1]))

        else:

            X = np.empty((self.batch_size, len(self.moda) * self.nb_samples,
                          SIZE[0], SIZE[1]))

        for i, ID in enumerate(list_IDs_temp):

            image = np.load(self.source + '/' + ID)
            indices = []
            indices_base = [a*self.nb_samples for a in self.moda]

            for base in indices_base:

                indices.extend(list(range(base, base+self.nb_samples)))

            image_moda = image[indices, :, :]

            X[i, ] = self.divise_aug(image_moda)

        X = X/255

        return X

    def divise_aug(self, image):

        if self.aug:

            image = self.augmentation(image)

        if self.divise:

            X = [image[self.nb_samples*i:self.nb_samples*(i+1), :, :]
                 for i in range(len(self.moda))]

        else:

            X = image

        return X

    def augmentation(self, im3d):

        im3d = self.flip(im3d)
        im3d = self.zoom(im3d)
        im3d = self.rotate(im3d).astype(np.uint8)

        bit_elas = random.getrandbits(1)

        if bit_elas:

            im3d = self.elastic(im3d)

        im3d = self.put_salt(im3d)

        return im3d.astype(np.uint8)

    def flip(self, image):

        bit = random.getrandbits(1)

        if bit:

            new_image = np.flip(image, axis=2)

        else:

            new_image = image

        return new_image

    def zoom(self, im3d):

        n = round(random.uniform(1, 1.2), 2)
        scale = iaa.Affine(scale=(n))

        new_image = np.zeros(im3d.shape)

        for i in range(im3d.shape[0]):

            img = im3d[i, :, :]
            img = scale.augment_image(img)
            new_image[i] = img

        return new_image

    def rotate(self, im3d):

        n = random.randint(-15, 15)
        rotation = iaa.geometric.Affine(rotate=n, mode="reflect")

        new_image = np.zeros(im3d.shape)

        for i in range(im3d.shape[0]):

            img = im3d[i, :, :]
            img = rotation.augment_image(img)
            new_image[i] = img

        return new_image

    def put_salt(self, im3d):

        n = np.round(random.uniform(0.00, 0.009), 3)

        salt = iaa.arithmetic.Salt(n)

        new_image = np.zeros(im3d.shape)

        for i in range(im3d.shape[0]):

            img = im3d[i, :, :]
            img = salt.augment_image(img)
            new_image[i] = img

        return new_image

    def elastic(self, im3d):

        alp = random.randint(40, 60)
        sig = random.randint(20, 40)

        new_image = np.zeros_like(im3d)

        elastic_tr = iaa.geometric.ElasticTransformation(alpha=alp,
                                                         sigma=sig,
                                                         mode="reflect")

        for i in range(im3d.shape[0]):

            img = im3d[i, :, :]
            img = elastic_tr.augment_image(img)
            new_image[i] = img

        return new_image.astype(np.uint8)

In [98]:
#########################
# Modelization function #
#########################

def modelize_gen(model, train_gen, val_gen, nb_epochs, batch, opti="adam",
                 verbose=0, graph=False):

    start = time()

    model.compile(loss='binary_crossentropy',
                  optimizer=opti,
                  metrics=['accuracy'])

    history = model.fit(train_gen,
                        batch_size=batch,
                        validation_data=val_gen,
                        epochs=nb_epochs,
                        verbose=verbose)

    train_acc = history.history["accuracy"]
    val_acc = history.history["val_accuracy"]
    train_loss = history.history["loss"]
    val_loss = history.history["val_loss"]

    if graph:

        fig = plt.figure(figsize=(10, 6))

        plt.subplot(2, 1, 1)
        plt.plot(range(1, nb_epochs+1), train_loss, 'b', label='Train loss',
                 color="green")
        plt.plot(range(1, nb_epochs+1), val_loss, 'b', label='Validation loss',
                 color="orange")
        plt.title('Train & validation losses - ')
        plt.xlabel("Epochs")
        plt.ylabel("Losses")
        plt.legend()

        plt.subplot(2, 1, 2)
        plt.plot(range(1, nb_epochs+1), train_acc, 'b', label='Train acc',
                 color="red")
        plt.plot(range(1, nb_epochs+1), val_acc, 'b', label='Validation acc',
                 color="blue")
        plt.title('Train & validation accuracies - ')
        plt.xlabel("Epochs")
        plt.ylabel("Accuracy")
        plt.legend()

        plt.tight_layout()
        plt.show()

        print()
        print(f"Temps d'entrainements pour {nb_epochs} epochs :\
        {time()-start:.0f} secondes.")
        print(f"Meilleure train_acc = {np.max(train_acc):.3f},\
        meilleure val_acc = {np.max(val_acc):.3f}")
        print()

    return model


##################
# Model creation #
##################

def create_model_4_12_150():

    inputs = Input((NB_MOD, NB_SAMPLES, SIZE[0], SIZE[1], 1))

    inA = inputs[:, 0, :, :, :]
    a = Conv3D(8, kernel_size=3, activation="relu")(inA)
    a = BatchNormalization()(a)
    a = Conv3D(8, kernel_size=3, activation="relu")(a)
    a = BatchNormalization()(a)
    a = MaxPool3D(pool_size=2)(a)
    a = Conv3D(16, kernel_size=3, activation="relu")(a)
    a = BatchNormalization()(a)
    a = MaxPool3D(pool_size=2)(a)
    a = Reshape((35, 35, 16))(a)
    a = Conv2D(32, kernel_size=3, activation="relu")(a)
    a = BatchNormalization()(a)
    a = MaxPool2D(pool_size=2)(a)
    a = Conv2D(32, kernel_size=3, activation="relu")(a)
    a = BatchNormalization()(a)
    a = MaxPool2D(pool_size=2)(a)
    a = Flatten()(a)

    inB = inputs[:, 1, :, :, :]
    b = Conv3D(8, kernel_size=3, activation="relu")(inB)
    b = BatchNormalization()(b)
    b = Conv3D(8, kernel_size=3, activation="relu")(b)
    b = BatchNormalization()(b)
    b = MaxPool3D(pool_size=2)(b)
    b = Conv3D(16, kernel_size=3, activation="relu")(b)
    b = BatchNormalization()(b)
    b = MaxPool3D(pool_size=2)(b)
    b = Reshape((35, 35, 16))(b)
    b = Conv2D(32, kernel_size=3, activation="relu")(b)
    b = BatchNormalization()(b)
    b = MaxPool2D(pool_size=2)(b)
    b = Conv2D(32, kernel_size=3, activation="relu")(b)
    b = BatchNormalization()(b)
    b = MaxPool2D(pool_size=2)(b)
    b = Flatten()(b)

    inC = inputs[:, 2, :, :, :]
    c = Conv3D(8, kernel_size=3, activation="relu")(inC)
    c = BatchNormalization()(c)
    c = Conv3D(8, kernel_size=3, activation="relu")(c)
    c = BatchNormalization()(c)
    c = MaxPool3D(pool_size=2)(c)
    c = Conv3D(16, kernel_size=3, activation="relu")(c)
    c = BatchNormalization()(c)
    c = MaxPool3D(pool_size=2)(c)
    c = Reshape((35, 35, 16))(c)
    c = Conv2D(32, kernel_size=3, activation="relu")(c)
    c = BatchNormalization()(c)
    c = MaxPool2D(pool_size=2)(c)
    c = Conv2D(32, kernel_size=3, activation="relu")(c)
    c = BatchNormalization()(c)
    c = MaxPool2D(pool_size=2)(c)
    c = Flatten()(c)

    inD = inputs[:, 3, :, :, :]
    d = BatchNormalization()(inD)
    d = Conv3D(8, kernel_size=3, activation="relu")(inD)
    d = BatchNormalization()(d)
    d = Conv3D(8, kernel_size=3, activation="relu")(d)
    d = BatchNormalization()(d)
    d = MaxPool3D(pool_size=2)(d)
    d = Conv3D(16, kernel_size=3, activation="relu")(d)
    d = BatchNormalization()(d)
    d = MaxPool3D(pool_size=2)(d)
    d = Reshape((35, 35, 16))(d)
    d = Conv2D(32, kernel_size=3, activation="relu")(d)
    d = BatchNormalization()(d)
    d = MaxPool2D(pool_size=2)(d)
    d = Conv2D(32, kernel_size=3, activation="relu")(d)
    d = BatchNormalization()(d)
    d = MaxPool2D(pool_size=2)(d)
    d = Flatten()(d)

    x = Concatenate()([a, b, c, d])

    outputs = Dense(1, activation="sigmoid")(x)

    model = Model(inputs=inputs, outputs=outputs)

    return model

In [100]:
##############
# submission #
##############

def submission(model, X_test, df_test):
    """
    Create the submission csv file
    """
    preds = model.predict(X_test)
    df_test = df_test[df_test["scan type"] == "FLAIR"]
    df_test["MGMT_value"] = list(np.squeeze(preds))
    df_test = df_test[["BraTS21ID", "MGMT_value"]]
    df_test.to_csv(OUT + "/" + "submission.csv", index=False)

In [ ]:
##########    
# Script #
##########

start = time()

print("Beginning submission script")
df = create_dataframe(TRAIN_PATH, MODS)
df_test = create_dataframe(TEST_PATH, MODS)

df = filter_train_df(df)
dico_labels = create_dicts(INPUT + "/" + "train_labels.csv")

IDs = df["BraTS21ID"].values.tolist()
IDs = sorted(list(set(IDs)))
train_IDs, val_IDs = train_test_split(IDs, test_size=0.15)

df_train = df[df["BraTS21ID"].isin(train_IDs)]
df_val = df[df["BraTS21ID"].isin(val_IDs)]

print("Dataframes created, beginning preprocessing")

preprocessing(df_train, OUT_TRAIN, MODS)
preprocessing(df_val, OUT_VAL, MODS)
preprocessing(df_test, OUT_TEST, MODS)

print("Preprocessing ended, preparing data generator")

# Train generator
liste_train = sorted(os.listdir(OUT_TRAIN))
random.shuffle(liste_train)
liste_train_labels = [dico_labels[elt.split(".")[0]] for elt in liste_train]

param_train = {"list_IDs" : liste_train * LIST_ITERATION, 
               "labels" : liste_train_labels * LIST_ITERATION,
               "source" : OUT_TRAIN, 
               "batch_size" : BATCH,
               "nb_samples" : 12, 
               "moda" : [0, 1, 2, 3],
               "shuffle" : True,
               "divise" : True,
               "aug" : True,
               "train" : True}

train_gen_4x12_150 = DataGenerator(**param_train)

# Validation generator
liste_val = sorted(os.listdir(OUT_VAL))
random.shuffle(liste_val)
liste_val_labels = [dico_labels[elt.split(".")[0]] for elt in liste_val]

param_val = {"list_IDs" : liste_val, 
               "labels" : liste_train_labels,
               "source" : OUT_VAL, 
               "batch_size" : BATCH,
               "nb_samples" : 12, 
               "moda" : [0, 1, 2, 3],
               "shuffle" : True,
               "divise" : True,
               "aug" : False,
               "train" : True}

val_gen_4x12_150 = DataGenerator(**param_val)

# Test generator
liste_test = sorted(os.listdir(OUT_TEST))
param_test = {"list_IDs" : liste_test,
             "labels" : None,
             "source" : OUT_TEST,
             "batch_size" : 1,
             "nb_samples" : 12,
             "moda" : [0, 1, 2, 3],
             "shuffle" : False,
             "aug" : False,
             "divise" : True,
             "train" : False}

test_gen_4x12_150 = DataGenerator(**param_test)

print("Beginning modelization")

model = create_model_4_12_150()

model = modelize_gen(model,
                     train_gen_4x12_150,
                     val_gen_4x12_150,
                     8,
                     batch = BATCH,
                     opti = "adam",
                     verbose = 1)

print("Model training ended, making submission")

submission(model, test_gen_4x12_150, df_test)

print(f"Submission process completed. It took {time()-start:.0f} seconds")

In [111]:
###################################################################
# BONUS : Find here two functions to test if the "zoi identifying #
# process" works, or not. It is needed to create the reference    #
# dataframes before using them. See demo below...                 #
###################################################################

def test_im(name, df_source):
    """
    Give the "BraTS21ID" of a patient and the dataframe (df_train, df_val
    or df_test) he belongs to.
    The function will return the "zoi" taken for the modelization.
    """
    row = df_source[(df_source["BraTS21ID"] == name) &
                    (df_source["scan type"] == "FLAIR")]

    path_dcm = row["chemins"].values[0]
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(path_dcm)
    reader.SetFileNames(dicom_names)
    im3D = reader.Execute()
    new_img3D = resample(im3D, ref_image)
    array_3D = normalize_img(sitk.GetArrayFromImage(new_img3D))
    array_3D = resize_3d(array_3D, SIZE_TEMP).astype(np.uint8)

    zoi = zoi_cube(array_3D, name)
    print(zoi)

    slices = zoi[5] - zoi[4]

    if slices < NB_SAMPLES:

        missing = NB_SAMPLES-slices

        extension = int(missing//2)
        lim_haute = zoi[4] - extension

        if missing % 2 == 0:

            lim_basse = zoi[5] + extension

        else:

            lim_basse = zoi[5] + extension + 1

        if lim_haute < 0:

            lim_haute = 0
            lim_basse = NB_SAMPLES

        if lim_basse > array_3D.shape[0] - 1:

            lim_basse = array_3D.shape[0] - 1
            lim_haute = array_3D.shape[0] - (NB_SAMPLES + 1)

    elif slices > NB_SAMPLES:

        surplus = slices - NB_SAMPLES

        reduction = int(surplus//2)
        lim_haute = zoi[4] + reduction

        if surplus % 2 == 0:

            lim_basse = zoi[5] - reduction

        else:

            lim_basse = zoi[5] - (reduction + 1)

    else:

        lim_haute = zoi[4]
        lim_basse = zoi[5]

    fig = plt.figure(figsize=(20, 42))

    for i in range(array_3D.shape[0]):

        if np.mean(array_3D[i]) != 0:
            mask = array_3D[i] > 0
            coords = np.argwhere(mask)

            y0, x0 = coords.min(axis=0)
            y1, x1 = coords.max(axis=0) + 1

            haut = y1-y0
            larg = x1-x0

        else:

            haut = 0
            larg = 0

        titre = f"ht {haut} {haut/SIZE_TEMP[0]:.2f} - la\
        {larg} {larg/SIZE_TEMP[0]:.2f}"

        dessin = plt.subplot(12, 4, i + 1)
        rect = patches.Rectangle((zoi[0], zoi[2]), 150, 150,
                                 linewidth=1.5, edgecolor='r',
                                 facecolor='none')

        if i >= lim_haute and i < lim_basse:

            dessin.add_patch(rect)

        dessin.title.set_text(titre)
        plt.imshow(array_3D[i, :, :])


def level_im(name, df_source):
    """
    Same as before but here the function shows the "bright flattened" image.
    """

    row = df_source[(df_source["BraTS21ID"] == name) &
                    (df_source["scan type"] == "FLAIR")]

    path_dcm = row["chemins"].values[0]
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(path_dcm)
    reader.SetFileNames(dicom_names)
    im3D = reader.Execute()
    new_img3D = resample(im3D, ref_image)
    array_3D = normalize_img(sitk.GetArrayFromImage(new_img3D))
    array_3D = resize_3d(array_3D, SIZE_TEMP).astype(np.uint8)

    rab = rabote(array_3D, 40)

    masked = np.ma.masked_equal(rab, 0)
    mean_image = np.mean(masked, axis=0).astype(np.uint8)

    eroded = cv2.erode(mean_image, None, iterations=5)
    eroded = normalize_img(eroded).astype(np.uint8)

    masked2 = np.ma.masked_equal(array_3D, 0)
    mean_image2 = np.mean(masked2, axis=0).astype(np.uint8)

    eroded2 = cv2.erode(mean_image2, None, iterations=5)
    eroded2 = normalize_img(eroded2).astype(np.uint8)

    fig = plt.figure(figsize=(6, 3))

    plt.subplot(1, 2, 1)
    plt.imshow(eroded2)
    plt.subplot(1, 2, 2)
    plt.imshow(eroded)

In [ ]:
# First create the dataframes

df = create_dataframe(TRAIN_PATH, MODS)
df_test = create_dataframe(TEST_PATH, MODS)

df = filter_train_df(df)
dico_labels = create_dicts(INPUT + "/" + "train_labels.csv")

IDs = df["BraTS21ID"].values.tolist()
IDs = sorted(list(set(IDs)))
train_IDs, val_IDs = train_test_split(IDs, test_size=0.15)

df_train = df[df["BraTS21ID"].isin(train_IDs)]
df_val = df[df["BraTS21ID"].isin(val_IDs)]

In [ ]:
# Now choose one dataframe and a "BraTS21ID" in it.
print(set(df_train["BraTS21ID"].values.tolist()[:100]))

In [ ]:
# And use a function with a "BraTS21ID".
# It will plot all the slices of a reoriented 3d image and show
# in the red squares what exactly was kept for
# modelizations...

name = "00022"
ref_df = df_train

test_im(name, ref_df)

In [ ]:
# Flattened version
level_im(name, ref_df)